In [1]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt

In [2]:

# Run authentication
ee.Authenticate()
ee.Initialize(project='dulcet-iterator-470310-n') # PUT YOUR API KEY (Project ID) HERE!

EEException: Project 'projects/dulcet-iterator-470310-n' not found or deleted.

# Spectral Indices
Today, we are going to expand on the lecture we had on Spectral Indices, and test some different approaches out in GEE. Typically, we will enact spectral indices on optical satellite data, specifically focusing on landsat and sentinel. However, it is possible to enact this on anything with bands that represent different reflectivities.

In this example, I will show you how to work with very high resolution data, and then you can test the Sentinel-2 and Landsat data on something you decide with different indices.


In [ ]:
## Importing some data
# Let's try importing some Sentinel 2 and some Landsat data for two different regions.
# We can also try some Planet satellite data, this is very high resolution.

# Import Sentinel 2
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Import Landsat 9
landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

# Import Planet data
planet = ee.ImageCollection("SKYSAT/GEN-A/PUBLIC/ORTHO/MULTISPECTRAL")

# Now that we have imported the data, we need to check where the planet data covers
# We know S2 and L9 are global data, so these are more available for you to work with.
# This is typically because aerial imagery covers small project areas of interested.

In [ ]:
# Calculate the footprint of the Planet data collection
planet_footprint = planet.geometry()

# Add the footprint to the map and display it
Map = geemap.Map()
Map.addLayer(planet_footprint, {}, 'Planet Footprint')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Define the Kanuku Mountains protected area in Guyana (example: a bounding box)
# You might want to refine this geometry for a more precise area of interest.
kanuku_mountains_area = ee.Geometry.Rectangle([-59.5, 2.5, -58.5, 3.5])

# Filter the Planet data by the Kanuku Mountains area
planet_kanuku = planet.filterBounds(kanuku_mountains_area)

# Get the list of images in the filtered collection
planet_kanuku_list = planet_kanuku.toList(planet_kanuku.size())

# Define visualization parameters
planet_vis_params = {
    'min': 0,
    'max': 4000,
    'bands': ['R', 'G', 'B'] # Assuming RGB bands are available
}

# Create a map object
Map = geemap.Map()
Map.centerObject(kanuku_mountains_area, 9) # Center map on the study area

# Iterate through the list of images and add each one as a layer
for i in range(planet_kanuku.size().getInfo()):
    image = ee.Image(planet_kanuku_list.get(i))
    date = image.date().format('YYYY-MM-dd').getInfo()
    Map.addLayer(image, planet_vis_params, f'Planet {date}')

Map

Map(center=[3.0000377833485774, -58.999999999999794], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
## Investigate the planet data architecture
planet_kanuku.size().getInfo()
planet_kanuku_list

# It seems the easiest way to define specific tiles we want is via the "catalogID property"

In [ ]:
## But, we only want some images without cloud for this analysis right?
# Define the specific catalog IDs to retrieve
specific_catalog_ids = [
    's02_20151005T143149Z',
    's02_20160118T145415Z',
    's01_20160307T145511Z'
]

# Filter the Planet data by the specific catalog IDs
# Use ee.Filter.inList('system:index', list_of_ids) to filter by system index (which is often the catalog ID)
planet_specific_images = planet.filter(ee.Filter.inList('system:index', specific_catalog_ids))

# Dictionary to store the images as variables
planet_images_by_catalogID = {}

print("Attempting to retrieve Planet images by catalogID and assign to variables:")

# Get the list of images in the filtered collection
planet_specific_images_list = planet_specific_images.toList(planet_specific_images.size())

# Check if the filtered collection is empty before iterating
if planet_specific_images.size().getInfo() > 0:
    # Iterate through the list of images and assign each one to a variable
    for i in range(planet_specific_images.size().getInfo()):
        image = ee.Image(planet_specific_images_list.get(i))
        catalog_id = image.get('system:index').getInfo()

        # Create a variable name based on the catalog ID
        var_name = f"planet_{catalog_id.replace('-', '_').replace('T', '_')}"
        planet_images_by_catalogID[var_name] = image
        print(f"Assigned Planet image with catalogID '{catalog_id}' to variable '{var_name}'")

    # Now, display the images on a map
    Map = geemap.Map()

    # Define visualization parameters (assuming RGB bands)
    planet_vis_params = {
        'min': 0,
        'max': 4000,
        'bands': ['R', 'G', 'B']
    }

    print("\nAttempting to display Planet images on the map:")

    # Iterate through the dictionary of images and add each to the map
    for var_name, image in planet_images_by_catalogID.items():
        # Use the catalog ID as the layer name for clarity
        layer_name = var_name.replace('planet_', '').replace('_', '-') # Convert back to a readable format
        Map.addLayer(image, planet_vis_params, f'Planet {layer_name}')
        print(f"Added layer '{f'Planet {layer_name}'}' to the map.")

    # Center the map on the first image (or a representative location if available)
    if planet_specific_images.size().getInfo() > 0:
        Map.centerObject(ee.Image(planet_specific_images_list.get(0)).geometry(), 9)

else:
    print("No Planet images found for the specified catalog IDs, so no map will be displayed.")

# Display the map
Map

Attempting to retrieve Planet images by catalogID and assign to variables:
Assigned Planet image with catalogID 's01_20160307T145511Z' to variable 'planet_s01_20160307_145511Z'
Assigned Planet image with catalogID 's02_20151005T143149Z' to variable 'planet_s02_20151005_143149Z'
Assigned Planet image with catalogID 's02_20160118T145415Z' to variable 'planet_s02_20160118_145415Z'

Attempting to display Planet images on the map:
Added layer 'Planet s01-20160307-145511Z' to the map.
Added layer 'Planet s02-20151005-143149Z' to the map.
Added layer 'Planet s02-20160118-145415Z' to the map.


Map(center=[3.0296657010276684, -59.339611046444034], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
## Time for some indices - it looks like we have bands:
# Blue (B), Green (G), Red (R), NIR (N), Panchromatic (P)
# Panchromatic allows us to interpolate a better resolution, but lets work with
# just the indices for now.
# First, lets define our variables as specific values

# Assuming the dictionary planet_images_by_catalogID was created in a previous cell and is available
if 'planet_images_by_catalogID' in locals():
    print("Defining variables for specific Planet images:")

    # Define the variables using the images from the dictionary
    if 'planet_s01_20160307_145511Z' in planet_images_by_catalogID:
        planet_s01_20160307_145511Z = planet_images_by_catalogID['planet_s01_20160307_145511Z']
        print("Defined variable: planet_s01_20160307_145511Z")
    else:
        print("Image with catalogID 's01_20160307T145511Z' not found in the dictionary.")

    if 'planet_s02_20151005_143149Z' in planet_images_by_catalogID:
        planet_s02_20151005_143149Z = planet_images_by_catalogID['planet_s02_20151005_143149Z']
        print("Defined variable: planet_s02_20151005_143149Z")
    else:
        print("Image with catalogID 's02_20151005T143149Z' not found in the dictionary.")

    if 'planet_s02_20160118_145415Z' in planet_images_by_catalogID:
        planet_s02_20160118_145415Z = planet_images_by_catalogID['planet_s02_20160118_145415Z']
        print("Defined variable: planet_s02_20160118_145415Z")
    else:
        print("Image with catalogID 's02_20160118T145415Z' not found in the dictionary.")

else:
    print("The dictionary 'planet_images_by_catalogID' was not found. Please ensure the cell that creates it has been executed.")

# You can now use these variables directly


Defining variables for specific Planet images:
Defined variable: planet_s01_20160307_145511Z
Defined variable: planet_s02_20151005_143149Z
Defined variable: planet_s02_20160118_145415Z


In [ ]:
# Time for some indices, it probably makes sense here to work with NDVI, or normalised
# difference vegetation index. This can allow us identify vegetation in an image.
# We do this with the calculation:
# NDVI = (NIR - RED)/(NIR + RED)

ndvi_20151005 = planet_s02_20151005_143149Z.normalizedDifference(['N', 'R']).rename('NDVI')
ndvi_2016_01_18 = planet_s02_20160118_145415Z.normalizedDifference(['N', 'R']).rename('NDVI')
ndvi_2016_03_07 = planet_s01_20160307_145511Z.normalizedDifference(['N', 'R']).rename('NDVI')

NDVIMap_kanuku = geemap.Map()
NDVIMap_kanuku.addLayer(ndvi_20151005, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI 2015-10-05')
NDVIMap_kanuku.addLayer(ndvi_2016_01_18, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI 2016-01-18')
NDVIMap_kanuku.addLayer(ndvi_2016_03_07, {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}, 'NDVI 2016-03-07')
NDVIMap_kanuku.centerObject(kanuku_mountains_area, 9)
NDVIMap_kanuku

Map(center=[3.0000377833485774, -58.999999999999794], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
## Time for some further analysis - we could try a a subtract function
diff_20151005_20160118 = ndvi_20151005.subtract(ndvi_2016_01_18)
diff_20160118_20160307 = ndvi_2016_01_18.subtract(ndvi_2016_03_07)

diffmap_kanuku = geemap.Map()

# Define visualization parameters with viridis palette
diff_vis_params = {
    'min': -1,
    'max': 1,
    'palette': 'RdYlGn' # Changed palette to viridis
}

diffmap_kanuku.addLayer(diff_20151005_20160118, diff_vis_params, 'Difference 2015-10-05 vs 2016-01-18')
diffmap_kanuku.addLayer(diff_20160118_20160307, diff_vis_params, 'Difference 2016-01-18 vs 2016-03-07')
diffmap_kanuku.add_colorbar(diff_vis_params, label="NDVI Difference", orientation="vertical")
diffmap_kanuku.centerObject(kanuku_mountains_area, 9)
diffmap_kanuku

Map(center=[3.0000377833485774, -58.999999999999794], controls=(WidgetControl(options=['position', 'transparen…

# Spectral Unmixing
**What is it?**
Spectral unmixing essentially breaks up reflectance signals in a band, allowing us to detect different things within the same band - this is then combined across bands to display a composite.

Think of an image as a musical chord - you hear the sound, but spectral unmixing attempts to determine how much of this response comes from each note.
The notes in this case are things such as rock, vegetation, water, snow, soil and so on. The result of spectral unmixing tells us that "maybe X, Y and Z percent of this pixel are vegetation, soil and rock, we will display the greatest proportion".

This is essentially an advanced index approach - a great way to test your skills in a project.

For further reference, check these:


*   https://developers.google.com/earth-engine/guides/image_transforms
*   https://developers.google.com/earth-engine/apidocs/ee-image-unmix (the ee.image toolbox also provides a lot of extra information on different transformations you can do with optical data)
*   https://step.esa.int/main/wp-content/help/versions/9.0.0/snap/org.esa.snap.snap.unmix.ui/unmixing/SpectralUnmixingTool.html



In [ ]:
## Let's also demonstrate spectral unmixing
# First we need to confirm the band names of the planet data.
planet_s01_20160307_145511Z.bandNames().getInfo()

# We do not want the P band, as it can cause issues with our output, as it is more for
# pan-sharpening than actual analysis here. Let's drop these bands.

# Get the list of all band names
all_bands_201603 = planet_s01_20160307_145511Z.bandNames()

# Create a list of bands to keep (all bands except 'P')
bands_to_keep = all_bands_201603.filter(ee.Filter.neq('item', 'P'))

# Select the bands to keep using the select() method
no_P_201510 = planet_s02_20151005_143149Z.select(bands_to_keep)
no_P_201601 = planet_s02_20160118_145415Z.select(bands_to_keep)
no_P_201603 = planet_s01_20160307_145511Z.select(bands_to_keep)


In [ ]:
# Continued with a removed P band
planet_bands = ['B', 'G', 'R', 'N']

# Define the endmembers (landsat 5 default, these can vary satellite to satellite)
urban = [88, 42, 48, 38]
veg = [50, 21, 20, 35]
water = [51, 20, 14, 9]

# Unmix each image
fractions_P201510 = no_P_201510.unmix([urban, veg, water])
fractions_P201601 = no_P_201601.unmix([urban, veg, water])
fractions_P201603 = no_P_201603.unmix([urban, veg, water])

# Create a map
fractionalmap = geemap.Map()
fractionalmap.addLayer(no_P_201601, planet_vis_params, 'base imagery 2016 01')
fractionalmap.addLayer(fractions_P201510, None, 'unmixed_201510')
fractionalmap.addLayer(fractions_P201601, None, 'unmixed_201601')
fractionalmap.addLayer(fractions_P201603, None, 'unmixed_201603')
fractionalmap.centerObject(kanuku_mountains_area, 9)
fractionalmap

Map(center=[3.0000377833485774, -58.999999999999794], controls=(WidgetControl(options=['position', 'transparen…

# Why was unmixing here unsuccessful?
Likely due to the type of terrain and notable difference between Landsat 5 and Planet reflectance. endmembers are things which need to be defined on a satellite to satellite basis - although they are often in the same general area. Therefore, a good way to find this answer is by pixel-checking different areas to see the reflectance in each band. This can be done through either sampling via polygon, or sampling via points. Something that is relatively easy to do in GIS, and in GEE. Give it a try.

Otherwise, for the rest of this session, try doing a similar analysis with S2 or Landsat, its often better to test Vegetation indices or a spectral unmixing approach on a bit more of a varied area - as this is simply an overview of a deep rainforest region.